In [13]:
# function utk kalkulasi fitness dari kromosom
def calculate_fitness(chromosome):
    total_weight = 0 # hitung total elemen pertama dari sebuah item
    total_value = 0 # hitung total elemen kedua dari sebuah item
    for i in range(len(chromosome)): # loop sepanjang kromosom
        if chromosome[i] == 1: # kalau diangkut
            total_weight += items[i]['berat'] # menambahkan berat item
            total_value += items[i]['nilai'] # menambahkan value item
    # print(total_weight)
    # memastikan hanya kromosom yang valid yang memiliki fitness
    # kromosom tidak valid jika melebihi kapasitas knapsack
    if total_weight > max_weight:
        return 0
    else:
        return total_value

In [14]:
import random

# funct utk generate random pop
def generate_population(size,length):
    # inisialisasi populasi sebagai list
    population = []
    # loop membangkitkan kromosom sejumlah ukuran populasi
    for i in range(size): # loop sejumlah banyaknya individu dalam populasi
        genes = [0, 1] # isi gen adalah 1 atau 0, 1 diangkut, 0 tidak
        chromosome = []
        for _ in range(length): # looping sebanyak item/barang
            # isi sebuah kromosom adalah 0 atau 1 sepanjang banyaknya barang
            chromosome.append(random.choice(genes)) # random memilih nol atau 1
        
        # membentuk individu yang terdiri dari kromosom dan fitness
        individu = {'chromosome' : chromosome,
                   'fitness' : calculate_fitness(chromosome)}
        # menambahkan individu baru ke dalam populasi
        population.append(individu)
    
    print("Generated a random population of size", size)
    return population

In [15]:
# funct utk memilih 2 kromosom yg akan crossover
import math

def select_chromosomes(population,crossover_probability):
    # memasukkan fitness setiap individu dalam populasi ke dalam list fitness_value
    fitness_values = []
    for i in range(len(population)):
        fitness_values.append(population[i]['fitness'])
    
    # sum(fitness_values) mentotal seluruh nilai list dalam fitness value
    probability = [float(fitness)/sum(fitness_values) for fitness in fitness_values]
    
    jumlah_kawin = math.ceil(crossover_probability * len(population))
    #print(jumlah_kawin)
    
    # ini akan memilih acak dari populasi sebanyak k parents dimana peluangnya sesuai
    # dengan besarnya nilai fitness dalam list fitness_values
    # misal probability isinya [0.5 0 0 0.3 0.2 0] maka item pertama 50% peluangnya, dst
    parents = random.choices(population, weights=probability, k=jumlah_kawin)
    
    return parents

In [16]:
# funct utk crossover 2 kromosom
def crossover(chromosome1,chromosome2):
    #membangkitkan bilangan random dari 2 sampai sepanjang chromosome-3, yaitu 2 hingga 2+6-4 yaitu 4
    crossover_point = random.randint(2, 2+len(chromosome1)-4)
    # anak1 : induk satu ke-0 sampai cross point-1 + induk2 dari cross point sampai habis
    # anak2 : induk dua ke-0 sampai cross point-1 + induk1 dari cross point sampai habis
    child1_chromosome = chromosome1[0:crossover_point] + chromosome2[crossover_point:]
    child2_chromosome = chromosome2[0:crossover_point] + chromosome1[crossover_point:]
    child1 = {'chromosome' : child1_chromosome,
             'fitness' : calculate_fitness(child1_chromosome)}
    child2 = {'chromosome' : child2_chromosome,
             'fitness' : calculate_fitness(child2_chromosome)}
    
    return child1, child2

In [17]:
# funct utk mutasi kromosom
def mutate(chromosome):
    #membangkitkan bilangan acak antara 0 hingga item-1
    for i in range(len(chromosome)):
        if random.uniform(0, 1) < mutation_probability: #jika hasil random di bawah prob mutasinya
            if chromosome[i] == 0:
                chromosome[i] = 1 # ubah dari 0 ke 1
            else:
                chromosome[i] = 0 # ubah dari 1 ke 0
    
    return chromosome

In [18]:
# urutkan pop desc
from operator import itemgetter

def sort_population(population):
    population = sorted(population, key=itemgetter('fitness'), reverse=True)
    return population

In [19]:
# menggunakan prinsip BGA untuk mempertahankan sejumlah tertentu individu dengan fitness tertinggi
def eliminate(population, top_population,count):
    # count adalah jumlah individu bernilai fitness tertinggi yang harus dipertahankan
    index = set()
    indexize = 0
    
    # mengacak sejumlah count bilangan yang harus dipastikan berbeda
    # misal count 4 maka membangkitkan 4 bilangan acak berbeda
    while indexize < count:
        # acak bilangan dari nol hingga size populasi -1 (misal size 10 maka 0 sampai 9)
        bil = random.randint(0, len(population)-1)
        if bil not in index:
            indexize += 1
            index.add(bil)
    
    # gantikan sejumlah jumlah_lestari kromosom dengan top_population
    j=0
    for i in index:
        # gantikan individu ke index tertentu dengan individu terbaik (j)
        population[i] = top_population[j]
        j += 1
    return population

In [20]:
# Tujuan adalah memaksimalkan nilai yang bisa diangkat
# Hard constraint: tidak boleh melebihi kapasitas alat angkut (max_weight)

# items yang dapat diangkut alat angkut
# sebuah item terdiri dari berat dan nilai
items = [
    {'berat' : 2,
     'nilai' : 2},
    {'berat' : 1,
     'nilai' : 4},
    {'berat' : 1,
     'nilai' : 3},
    {'berat' : 2,
     'nilai' : 5},
    {'berat' : 4,
     'nilai' : 7},
    {'berat' : 3,
     'nilai' : 9},
    {'berat' : 3,
     'nilai' : 5},
    {'berat' : 4,
     'nilai' : 10},
    {'berat' : 1,
     'nilai' : 1},
    {'berat' : 2,
     'nilai' : 4}
]
# print available items
print('Available items:\n', items)
max_weight = 16
print('Max weight:', max_weight)

# parameters for genetic algorithm
population_size = 100
crossover_probability = 0.8
mutation_probability = 0.1
generations = 100
r_bga = 0.1
jumlah_lestari = int(r_bga * population_size)

print('\nGenetic algorithm parameters:')
print('Population:', population_size)
print('Mutation probability:', mutation_probability)
print('Crossover probability:', crossover_probability)
print('Generations:', generations)
print('Value of r for BGA: {} sehingga jumlah individu yang dilestarikan {}'
      .format(r_bga,jumlah_lestari))

print('Performing genetic evolution:')

# generate a random population
population = generate_population(population_size,len(items))
# mengurutkan populasi berdasarkan fitness descending
population = sort_population(population)
# mengambil top population sesuai jumlah yang dilestarikan
top_population = population[0:(jumlah_lestari+1)]

population

Available items:
 [{'berat': 2, 'nilai': 2}, {'berat': 1, 'nilai': 4}, {'berat': 1, 'nilai': 3}, {'berat': 2, 'nilai': 5}, {'berat': 4, 'nilai': 7}, {'berat': 3, 'nilai': 9}, {'berat': 3, 'nilai': 5}, {'berat': 4, 'nilai': 10}, {'berat': 1, 'nilai': 1}, {'berat': 2, 'nilai': 4}]
Max weight: 16

Genetic algorithm parameters:
Population: 100
Mutation probability: 0.1
Crossover probability: 0.8
Generations: 100
Value of r for BGA: 0.1 sehingga jumlah individu yang dilestarikan 10
Performing genetic evolution:
Generated a random population of size 100


[{'chromosome': [0, 0, 1, 1, 0, 1, 1, 1, 1, 1], 'fitness': 37},
 {'chromosome': [0, 1, 0, 0, 1, 1, 1, 1, 1, 0], 'fitness': 36},
 {'chromosome': [0, 1, 0, 0, 1, 1, 1, 1, 1, 0], 'fitness': 36},
 {'chromosome': [0, 0, 0, 1, 1, 1, 0, 1, 1, 1], 'fitness': 36},
 {'chromosome': [0, 0, 0, 0, 1, 1, 1, 1, 0, 1], 'fitness': 35},
 {'chromosome': [1, 1, 1, 0, 1, 1, 0, 1, 0, 0], 'fitness': 35},
 {'chromosome': [0, 1, 1, 0, 0, 1, 1, 1, 0, 1], 'fitness': 35},
 {'chromosome': [0, 1, 0, 1, 1, 1, 0, 1, 0, 0], 'fitness': 35},
 {'chromosome': [0, 0, 0, 0, 1, 1, 1, 1, 0, 1], 'fitness': 35},
 {'chromosome': [1, 0, 1, 1, 0, 1, 1, 1, 0, 0], 'fitness': 34},
 {'chromosome': [1, 0, 1, 0, 0, 1, 1, 1, 1, 1], 'fitness': 34},
 {'chromosome': [0, 0, 1, 0, 1, 1, 1, 1, 0, 0], 'fitness': 34},
 {'chromosome': [0, 1, 0, 1, 1, 1, 1, 0, 0, 1], 'fitness': 34},
 {'chromosome': [0, 1, 0, 0, 0, 1, 1, 1, 0, 1], 'fitness': 32},
 {'chromosome': [1, 1, 1, 0, 1, 0, 1, 1, 1, 0], 'fitness': 32},
 {'chromosome': [1, 1, 1, 1, 1, 1, 0, 0,

In [21]:
# melakukan pembangkitan setiap generasi
for i in range(generations):
    print('\n', 'Generations:', i+1)
    
    # menentukan jumlah perkawinan yaitu jumlah anak dikali 2 (bulatkan ke atas)
    # misal jumlah anak 5 maka jumlah kawin adalah ceil(5/2) = 3 perkawinan
    # pilih induk
    # pilih induk
    parents = select_chromosomes(population,crossover_probability)
    
    # hapus parents dari populasi
    for par in range(len(parents)):
        for popu in range(len(population)):
            if parents[par] == population[popu]:
                population.pop(popu) # keluarkan parent terpilih dari populasi
                break # keluar dari for kedua
    
    # kawin silang
    for cross in range(len(parents)//2):
        # mengawinkan parent 0 dgn 1, kemudian 2 dgn 3 dan seterusnya sampai habis
        child1, child2 = crossover(parents[0+2*cross]['chromosome'],parents[1+2*cross]['chromosome'])
        
        # mutasi
        child1['chromosome'] = mutate(child1['chromosome'])
        child2['chromosome'] = mutate(child2['chromosome'])
        
        # menghitung fitness anak setelah dimutasi
        child1['fitness'] = calculate_fitness(child1['chromosome'])
        child2['fitness'] = calculate_fitness(child2['chromosome'])
        
        # menghitung fitness anak setelah dimutasi anak ke dalam generasi
        population.append(child1)
        population.append(child2)
    
    # BGA
    # melaksanakan proses BGA (menggantikan beberapa individu acak dgn list individu terbaik)
    population = eliminate(population, top_population, jumlah_lestari)
    # mengurutkan kembali populasi secara descending berdasarkan fitness
    population = sort_population(population)
    # pastikan ukuran populasi tetap 100
    population = population[0:100]
    # menyimpan individu terbaik sejumlah testari
    top_population = population[0:(jumlah_lestari+1)]
    
    # mencetak satu individu terbaik pada setiap generasi
    print('Best Individu:', population[0])

population


 Generations: 1
Best Individu: {'chromosome': [0, 0, 1, 1, 0, 1, 1, 1, 1, 1], 'fitness': 37}

 Generations: 2
Best Individu: {'chromosome': [0, 1, 1, 1, 1, 1, 0, 1, 1, 0], 'fitness': 39}

 Generations: 3
Best Individu: {'chromosome': [0, 1, 1, 1, 1, 1, 0, 1, 1, 0], 'fitness': 39}

 Generations: 4
Best Individu: {'chromosome': [0, 1, 1, 1, 1, 1, 0, 1, 1, 0], 'fitness': 39}

 Generations: 5
Best Individu: {'chromosome': [0, 1, 1, 1, 0, 1, 1, 1, 0, 1], 'fitness': 40}

 Generations: 6
Best Individu: {'chromosome': [0, 1, 1, 1, 0, 1, 1, 1, 0, 1], 'fitness': 40}

 Generations: 7
Best Individu: {'chromosome': [0, 1, 1, 1, 0, 1, 1, 1, 0, 1], 'fitness': 40}

 Generations: 8
Best Individu: {'chromosome': [0, 1, 1, 1, 0, 1, 1, 1, 0, 1], 'fitness': 40}

 Generations: 9
Best Individu: {'chromosome': [0, 1, 1, 1, 0, 1, 1, 1, 0, 1], 'fitness': 40}

 Generations: 10
Best Individu: {'chromosome': [0, 1, 1, 1, 0, 1, 1, 1, 0, 1], 'fitness': 40}

 Generations: 11
Best Individu: {'chromosome': [0, 1, 1, 1

[{'chromosome': [0, 1, 1, 1, 0, 1, 1, 1, 0, 1], 'fitness': 40},
 {'chromosome': [0, 1, 1, 1, 0, 1, 1, 1, 0, 1], 'fitness': 40},
 {'chromosome': [0, 1, 1, 1, 0, 1, 1, 1, 0, 1], 'fitness': 40},
 {'chromosome': [0, 1, 1, 1, 0, 1, 1, 1, 0, 1], 'fitness': 40},
 {'chromosome': [0, 1, 1, 1, 0, 1, 1, 1, 0, 1], 'fitness': 40},
 {'chromosome': [0, 1, 1, 1, 0, 1, 1, 1, 0, 1], 'fitness': 40},
 {'chromosome': [0, 1, 1, 1, 0, 1, 1, 1, 0, 1], 'fitness': 40},
 {'chromosome': [0, 1, 1, 1, 0, 1, 1, 1, 0, 1], 'fitness': 40},
 {'chromosome': [0, 1, 1, 1, 0, 1, 1, 1, 0, 1], 'fitness': 40},
 {'chromosome': [0, 1, 1, 1, 0, 1, 1, 1, 0, 1], 'fitness': 40},
 {'chromosome': [0, 1, 1, 1, 0, 1, 1, 1, 0, 1], 'fitness': 40},
 {'chromosome': [0, 1, 1, 1, 0, 1, 1, 1, 0, 1], 'fitness': 40},
 {'chromosome': [0, 1, 1, 1, 0, 1, 1, 1, 0, 1], 'fitness': 40},
 {'chromosome': [0, 1, 0, 1, 0, 1, 1, 1, 1, 1], 'fitness': 38},
 {'chromosome': [1, 1, 1, 1, 0, 1, 0, 1, 0, 1], 'fitness': 37},
 {'chromosome': [1, 1, 1, 1, 0, 1, 0, 1,

In [23]:
# Q1
# funct utk tampilkan barang yang diangkut
def display_selected_items(best_chromosome):
    print("Barang-barang yang diangkut:")
    
    total_weight = 0
    total_value = 0
    
    print("\nNo. | Berat | Nilai | Status")
    print("-" * 30)
    
    for i in range(len(best_chromosome)):
        status = "Diangkut" if best_chromosome[i] == 1 else "Tidak diangkut"
        print(f"{i+1:2d}  | {items[i]['berat']:5d} | {items[i]['nilai']:5d} | {status}")
        
        if best_chromosome[i] == 1:
            total_weight += items[i]['berat']
            total_value += items[i]['nilai']
    
    print(f"Total berat barang yang diangkut: {total_weight}")
    print(f"Total nilai barang yang diangkut: {total_value}")
    print(f"Kapasitas maksimum: {max_weight}")

print("\nProses evolusi genetika selesai.")
best_solution = population[0]  # Individu terbaik adalah yang di posisi pertama (indeks 0)
print(f"Solusi terbaik memiliki fitness: {best_solution['fitness']}")

# Memanggil function untuk menampilkan barang yang diangkut
display_selected_items(best_solution['chromosome'])


Proses evolusi genetika selesai.
Solusi terbaik memiliki fitness: 40
Barang-barang yang diangkut:

No. | Berat | Nilai | Status
------------------------------
 1  |     2 |     2 | Tidak diangkut
 2  |     1 |     4 | Diangkut
 3  |     1 |     3 | Diangkut
 4  |     2 |     5 | Diangkut
 5  |     4 |     7 | Tidak diangkut
 6  |     3 |     9 | Diangkut
 7  |     3 |     5 | Diangkut
 8  |     4 |    10 | Diangkut
 9  |     1 |     1 | Tidak diangkut
10  |     2 |     4 | Diangkut
Total berat barang yang diangkut: 16
Total nilai barang yang diangkut: 40
Kapasitas maksimum: 16
